In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score


In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [3]:
df_orig = pd.read_csv('full_dataset_unscaled.csv')

In [4]:
df_orig = df_orig.rename(columns={"datetime":"year-month"})

In [5]:
df_orig['net_job_rate']=df_orig['job_creation_rate']-df_orig['job_destruction_rate']
df_orig['apartment_for_rent_searches'] = df_orig[['apartment for rent','studio for rent','1 bedroom for rent','3 bedroom for rent']].sum(axis=1)
df_orig['multifamily_for_rent_searches'] = df_orig[['townhomes for rent','townhouse for rent','house for rent','duplex apartments for rent','condos for rent']].sum(axis=1)
df_orig['gun_searches'] = df_orig[['gun range', 'gun control', 'gun violence']].sum(axis=1)

In [6]:
df_orig['zri'] = np.log(df_orig['zri'])

In [7]:
gen_cols = ['zip',
 'City',
 'State',
 'Metro',
 'CountyName',
 'zri',
 'year',
 'month',
 'year-month']
acs_cols = ['percent_white',
 'percent_black',
 'percent_asian',
 'percent_hispanic',
 'percent_native_am',
 'percent_other_race',
 'percent_0_17',
 'percent_18_39',
 'percent_40_64',
 'percent_65+',
 'percent_rental_units_vacant',
 'percent_rental_units_occupied',
 'percent_graduate_deg',
 'percent_bachelors',
 'percent_associates',
 'percent_highschool',
 'percent_less_highschool',
 'percent_commute_public_transport',
 'percent_commute_less_30',
 'percent_buildings_less_10_units',
 'percent_buildings_10_19_units',
 'percent_buildings_20_49_units',
 'percent_buildings_50+_units',
 'percent_commute_30_to_59',
 'percent_commute_60_to_89',
 'percent_commute_90_more',
 'percent_new_city',
 'percent_new_unit',
 'percent_units_owner_occupied',
 'median_building_age',
 'income_per_capita',
 'poverty_rate',
 'total_pop',
 'percent_workforce_unemployed',
 'percent_work_from_home',
 'median_age',
 'percent_female',
 'gini_index',
 'percent_not_us_citizen']
acs_cols_remove=['percent_other_race','percent_40_64','percent_0_17','percent_18_39',
       'percent_65+','percent_rental_units_vacant','percent_not_us_citizen','percent_less_highschool', 'percent_buildings_less_10_units',
 'percent_commute_30_to_59',
 'percent_commute_60_to_89',
 'percent_commute_90_more', 'percent_commute_less_30','percent_graduate_deg',
                'percent_female','gini_index','percent_hispanic','percent_black','percent_bachelors',
                 'percent_asian','percent_new_city','percent_new_unit']
acs_cols_keep=list(set(acs_cols) - set(acs_cols_remove))
bikeshare_cols = ['bs_total_stations',
 'bs_total_systems',
 'has_bike_sharing']
bikeshare_cols_remove=['has_bike_sharing','bs_total_systems']
bikeshare_cols_keep=list(set(bikeshare_cols) - set(bikeshare_cols_remove))
trends_cols = ['gun range',
 'gun control',
 'gun violence',
 'job opportunities',
 'unemployment',
 'retirement',
 'layoff',
 'lgbt',
 'same sex marriage',
 'they',
 'pronouns',
 'black lives matter',
 'political correctness',
 'make america great again',
 'euthanasia',
 'getaway',
 'places to go',
 'flight tickets',
 'twitter',
 'hashtag',
 'fake news',
 'hurricane',
 'wildfire',
 'flood',
 'fire',
 "trader joe's",
 'whole foods',
 'lululemon',
 'thrift',
 'condos for rent',
 'duplex apartments for rent',
 'townhomes for rent',
 'townhouses for rent',
 'home for rent',
 'house for rent',
 'townhome for rent',
 'townhouse for rent',
 'apartment for rent',
 'studio for rent',
 '1 bedroom for rent',
 '3 bedroom for rent',
 'starbucks',
  'apartment_for_rent_searches',
  'multifamily_for_rent_searches',
  'gun_searches']
trends_cols_remove=['they','apartment for rent','studio for rent','1 bedroom for rent',
                    '3 bedroom for rent', 'townhome for rent','townhouse for rent','townhomes for rent',
                    'townhouses for rent','house for rent','home for rent','duplex apartments for rent','condos for rent',
                   'gun range', 'gun control', 'gun violence','homes for rent']
trends_cols_keep=list(set(trends_cols) - set(trends_cols_remove))
economic_cols = ['total_firms',
 'job_creation_rate',
 'job_destruction_rate',
 'startup_firms','state_local_perc', 'net_job_rate']
economic_cols_remove=['total_firms', 'job_creation_rate','job_destruction_rate',]
economic_cols_keep=list(set(economic_cols) - set(economic_cols_remove))

In [8]:
for zipcode in df_orig['zip'].unique():
    globals()[f"scaler_{zipcode}"]=StandardScaler(copy=False)
    df_filtered=df_orig[df_orig['zip']==zipcode]
    df_filtered_train = df_filtered[df_filtered['year']<2019]
    globals()[f"scaler_{zipcode}"].fit(df_filtered_train[['zri']])
    df_orig.loc[df_orig['zip']==zipcode,'zri']=globals()[f"scaler_{zipcode}"].transform(df_filtered[['zri']])

In [9]:
df_orig = df_orig[gen_cols + acs_cols_keep + bikeshare_cols_keep + economic_cols_keep + trends_cols_keep]

In [10]:
scale_columns = ['percent_buildings_50+_units', 'percent_associates',
       'percent_rental_units_occupied', 'percent_white', 'percent_highschool',
       'percent_work_from_home', 'percent_buildings_20_49_units',
       'median_building_age', 'median_age', 'percent_commute_public_transport',
       'percent_buildings_10_19_units', 'income_per_capita',
       'percent_native_am', 'percent_workforce_unemployed', 'poverty_rate',
       'percent_units_owner_occupied', 'total_pop', 'bs_total_stations',
       'startup_firms', 'state_local_perc', 'net_job_rate', 'gun_searches',
       'wildfire', 'fire', 'lgbt', 'political correctness', 'lululemon',
       'make america great again', 'same sex marriage', 'job opportunities',
       'retirement', 'black lives matter', 'flight tickets', 'pronouns',
       'trader joe\'s', 'fake news', 'hurricane', 'flood', 'whole foods',
       'twitter', 'thrift', 'hashtag', 'apartment_for_rent_searches', 'layoff',
       'starbucks', 'getaway', 'places to go', 'unemployment', 'euthanasia',
       'multifamily_for_rent_searches']

In [11]:
for zipcode in df_orig['zip'].unique():
    globals()[f"scaler_features_{zipcode}"]=StandardScaler(copy=False)
    df_filtered=df_orig[df_orig['zip']==zipcode]
    df_filtered_train = df_filtered[df_filtered['year']<2019]
    globals()[f"scaler_features_{zipcode}"].fit(df_filtered_train[scale_columns])
    df_orig.loc[df_orig['zip']==zipcode,scale_columns]=globals()[f"scaler_features_{zipcode}"].transform(df_filtered[scale_columns])

In [12]:
df_scaled_no_lags = df_orig.copy()

In [14]:
df_zri = df_scaled_no_lags[gen_cols]


In [ ]:
df_all_ext = df_scaled_no_lags

In [15]:
def laggenerator(i,colname,df):
    timelist=list(df['year-month'].drop_duplicates().sort_values())[0:i]
    df.loc[:,f'{colname}_lag{i}']=df.loc[:,f'{colname}'].shift(i)
    df.loc[df['year-month'].isin(timelist),f'{colname}_lag{i}']=0
    return df

def laggenerator_diff(i,colname,df):
    timelist=list(df['year-month'].drop_duplicates().sort_values())[0:i+1]
    df.loc[:,f'{colname}_diff_lag{i}_lag{i+1}']=df.loc[:,f'{colname}'].shift(i)-df.loc[:,f'{colname}'].shift(i+1)
    df.loc[df['year-month'].isin(timelist),f'{colname}_diff_lag{i}_lag{i+1}']=0
    return df

def laggenerator_diff12(i,colname,df):
    timelist=list(df['year-month'].drop_duplicates().sort_values())[0:i+11]
    df.loc[:,f'{colname}_diff_lag{i}_lag{i+11}']=df.loc[:,f'{colname}'].shift(i)-df.loc[:,f'{colname}'].shift(i+11)
    df.loc[df['year-month'].isin(timelist),f'{colname}_diff_lag{i}_lag{i+11}']=0
    return df

#ZRI 
def lag_gen(df):
    for i in range(1,12):
        df=laggenerator(i, 'zri', df)
    df=laggenerator_diff12(1, 'zri', df)
    return df

In [17]:
df_zri = lag_gen(df_zri)

# 1. AUTO (JUST ZRI) 

In [18]:
df = df_zri
train = df[(df['year'] < 2019) & (df['year']>=2015)]
test = df[df['year']==2019]
save_train = train.copy()
save_test = test.copy()

train = train.drop(['zip','City','State','Metro','CountyName','year','month','year-month'],axis=1)
train_y = train['zri']
train_X = train.drop(['zri'],axis=1)

test_y = test['zri']
test_X = test
#test_X = test.drop(['zri'],axis=1)

In [19]:
#Lasso Grid Search
lasso = Lasso()
grid = dict()
grid['alpha'] = [1e-15, 1e-10, 1e-8, 1e-5,1e-4, 1e-3,1e-2, 1, 5, 10]
lasso_grid = GridSearchCV(lasso, grid, cv=5, n_jobs=-1).fit(train_X,train_y)
lasso_best = lasso_grid.best_estimator_
print(f'lasso_best : {lasso_best}')

lasso_best.fit(train_X, train_y)


lasso_best : Lasso(alpha=1e-05)


Lasso(alpha=1e-05)

In [20]:
predictor_table=df.copy()
X_test = test_X.copy()

#loop through all month in 2019
for month in list(X_test['year-month'].drop_duplicates()):
    
    #run prediction for one month
    X_test=X_test[X_test['year-month']==month]
    X_test=X_test.drop(['zip','year','month','City','State','Metro','CountyName','year-month','zri'],axis=1)
    val=lasso_best.predict(X_test)

    #write current month prediction into predictor_table

    predictor_table.loc[predictor_table['year-month']==month,'zri']=val
        
    predictor_table=lag_gen(predictor_table)

    X_test=predictor_table[predictor_table['year']==2019]


In [22]:
scaled_predictions_y = X_test['zri']
temp = pd.concat([save_test['zip'],test_y],axis=1)
temp.reset_index(drop=True, inplace=True)
scaled_predictions_y.reset_index(drop=True, inplace=True)
rstable = pd.concat([temp,scaled_predictions_y],axis=1)
rstable.columns = ['zip','zri_test','zri_predicted']
rstable

,zip,zri_test,zri_predicted
0,1013,0.973309,1.062795
1,1013,0.960762,1.051512
2,1013,1.060825,1.056004
3,1013,1.160167,1.065848
4,1013,1.184891,1.073373
...,...,...,...
15607,99654,0.007888,1.066825
15608,99654,-0.102332,1.041338
15609,99654,0.407646,1.023838
15610,99654,-0.277916,1.014662


In [23]:
for zipcode in rstable['zip'].unique():
    rstable_filtered=rstable[rstable['zip']==zipcode]
    rstable.loc[rstable['zip']==zipcode,'zri_test']=globals()[f"scaler_{zipcode}"].inverse_transform(rstable_filtered[['zri_test']])
    rstable.loc[rstable['zip']==zipcode,'zri_predicted']=globals()[f"scaler_{zipcode}"].inverse_transform(rstable_filtered[['zri_predicted']])  
rstable.loc[:,'zri_test'] = np.exp(rstable.loc[:,'zri_test'])
rstable.loc[:,'zri_predicted'] = np.exp(rstable.loc[:,'zri_predicted'])
rstable.isna().sum()

zip              0
zri_test         0
zri_predicted    0
dtype: int64

In [24]:
#RESULTS 
r2 = r2_score(rstable['zri_test'],rstable['zri_predicted'])
rmse = sqrt(mean_squared_error(rstable['zri_test'],rstable['zri_predicted']))
print(f'R2: {r2*100}')
print(f'RMSE: {rmse}')

R2: 98.93213838217116
RMSE: 68.75143047884802


In [25]:
train_X.columns

Index(['zri_lag1', 'zri_lag2', 'zri_lag3', 'zri_lag4', 'zri_lag5', 'zri_lag6',
       'zri_lag7', 'zri_lag8', 'zri_lag9', 'zri_lag10', 'zri_lag11',
       'zri_diff_lag1_lag12'],
      dtype='object')

In [26]:
importance = np.abs(lasso_best.coef_)
coef = lasso_best.coef_
feature_names = train_X.columns
feature_importances= pd.DataFrame([feature_names,importance,coef]).T
feature_importances.columns = ['feature_names','coef_abs','coef']

feature_importances = feature_importances.sort_values(by='coef_abs',ascending=False)

In [27]:
feature_importances['Parent_feature']=feature_importances['feature_names'].apply(lambda x: x[0:x.find('_lag')] if x.find('_lag')!=-1 else x)
feature_importances['Parent_feature']=feature_importances['Parent_feature'].apply(lambda x: x[0:x.find('_diff')] if x.find('_diff')!=-1 else x)

In [28]:
feature_importances

,feature_names,coef_abs,coef,Parent_feature
0,zri_lag1,1.527279,1.527279,zri
1,zri_lag2,0.613722,-0.613722,zri
2,zri_lag3,0.059676,-0.059676,zri
11,zri_diff_lag1_lag12,0.038757,0.038757,zri
6,zri_lag7,0.032243,0.032243,zri
3,zri_lag4,0.031968,0.031968,zri
10,zri_lag11,0.026219,0.026219,zri
9,zri_lag10,0.014151,0.014151,zri
7,zri_lag8,0.012129,-0.012129,zri
8,zri_lag9,0.011088,0.011088,zri


In [ ]:
# title = 'Final_Auto'
# fi = title+'_FI'
# rstable.to_csv(f'../../../zillow_orientation/Residuals/{title}.csv')
# feature_importances.to_csv(f'../../../zillow_orientation/Residuals/{fi}.csv')

In [ ]:
# BEST AUTOREGRESSIVE 
# 12 Features total 
# ['zri_lag1', 'zri_lag2', 'zri_lag3', 'zri_lag4', 'zri_lag5', 'zri_lag6',
#        'zri_lag7', 'zri_lag8', 'zri_lag9', 'zri_lag10', 'zri_lag11',
#        'zri_diff_lag1_lag12']
# Lasso(alpha=1e-05)
# R2: 98.93213838217116
# RMSE: 68.75143047884802
# top coefs: zri_lag1,zri_lag2,zri_lag3,zri_diff_lag1_lag12



# OLD TRIALS TO IGNORE
# ['zri_lag1', 'zri_lag6', 'zri_diff_lag1_lag12']
# R2: 98.85027227385696
# RMSE: 71.3381352101476
    
# ['zri_lag1', 'zri_lag2', 'zri_lag3', 'zri_lag4', 'zri_lag5', 'zri_lag6',
#        'zri_lag7', 'zri_lag8', 'zri_lag9', 'zri_lag10', 'zri_lag11',
#        'zri_lag12', 'zri_diff_lag1_lag12']   
# R2: 98.93199923119373
# RMSE: 68.7559097657487
    
# ['zri_lag1', 'zri_diff_lag1_lag2', 'zri_lag2', 'zri_diff_lag2_lag3',
#        'zri_lag3', 'zri_diff_lag3_lag4', 'zri_lag4', 'zri_diff_lag4_lag5',
#        'zri_lag5', 'zri_diff_lag5_lag6', 'zri_lag6', 'zri_diff_lag6_lag7',
#        'zri_lag7', 'zri_diff_lag7_lag8', 'zri_lag8', 'zri_diff_lag8_lag9',
#        'zri_lag9', 'zri_diff_lag9_lag10', 'zri_lag10', 'zri_diff_lag10_lag11',
#        'zri_lag11', 'zri_diff_lag11_lag12', 'zri_diff_lag1_lag12']
# R2: 98.93217535166379
# RMSE: 68.75024037731302

# 2. ZRI + ACS 

In [76]:
df_zri_acs = df_scaled_no_lags[gen_cols+acs_cols_keep]

In [77]:
df_zri_acs = lag_gen(df_zri_acs)

/Users/nikiagrawal/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/Users/nikiagrawal/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/Users/nikiagrawal/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

In [78]:
for col in acs_cols_keep:
    df_zri_acs = laggenerator_diff12(1,col,df_zri_acs)
    df_zri_acs = laggenerator(1,col,df_zri_acs)
    df_zri_acs = laggenerator(6,col,df_zri_acs)
    df_zri_acs = laggenerator(12,col,df_zri_acs)
    #for i in range(1,4):
       # df_zri_acs = laggenerator_diff(i,col,df)


In [79]:
df = df_zri_acs
train = df[(df['year'] < 2019) & (df['year']>=2015)]
test = df[df['year']==2019]
save_train = train.copy()
save_test = test.copy()

train = train.drop(['zip','City','State','Metro','CountyName','year','month','year-month'],axis=1)
train_y = train['zri']
train_X = train.drop(['zri'],axis=1)

test_y = test['zri']
test_X = test
#test_X = test.drop(['zri'],axis=1)

In [81]:
#Lasso Grid Search
lasso = Lasso()
grid = dict()
grid['alpha'] = [1e-15, 1e-10, 1e-8, 1e-5,1e-4, 1e-3,1e-2, 1, 5, 10]
lasso_grid = GridSearchCV(lasso, grid, cv=5, n_jobs=-1).fit(train_X,train_y)
lasso_best = lasso_grid.best_estimator_
print(f'lasso_best : {lasso_best}')

lasso_best.fit(train_X, train_y)


lasso_best : Lasso(alpha=0.0001)


Lasso(alpha=0.0001)

In [82]:
predictor_table=df.copy()
X_test = test_X.copy()

#loop through all month in 2019
for month in list(X_test['year-month'].drop_duplicates()):
    
    #run prediction for one month
    X_test=X_test[X_test['year-month']==month]
    X_test=X_test.drop(['zip','year','month','City','State','Metro','CountyName','year-month','zri'],axis=1)
    val=lasso_best.predict(X_test)

    #write current month prediction into predictor_table

    predictor_table.loc[predictor_table['year-month']==month,'zri']=val
        
    predictor_table=lag_gen(predictor_table)

    X_test=predictor_table[predictor_table['year']==2019]


In [83]:
scaled_predictions_y = X_test['zri']
temp = pd.concat([save_test['zip'],test_y],axis=1)
temp.reset_index(drop=True, inplace=True)
scaled_predictions_y.reset_index(drop=True, inplace=True)
rstable = pd.concat([temp,scaled_predictions_y],axis=1)
rstable.columns = ['zip','zri_test','zri_predicted']
rstable

,zip,zri_test,zri_predicted
0,1013,0.973309,1.072649
1,1013,0.960762,1.077547
2,1013,1.060825,1.099619
3,1013,1.160167,1.126671
4,1013,1.184891,1.149418
...,...,...,...
15607,99654,0.007888,1.202899
15608,99654,-0.102332,1.213981
15609,99654,0.407646,1.235056
15610,99654,-0.277916,1.266095


In [84]:
# scaled_predictions_y = pd.Series(lasso_best.predict(test_X))
# temp = pd.concat([save_test['zip'],test_y],axis=1)
# temp.reset_index(drop=True, inplace=True)
# scaled_predictions_y.reset_index(drop=True, inplace=True)
# rstable = pd.concat([temp,scaled_predictions_y],axis=1)
# rstable.columns = ['zip','zri_test','zri_predicted']
# rstable

In [85]:
for zipcode in rstable['zip'].unique():
    rstable_filtered=rstable[rstable['zip']==zipcode]
    rstable.loc[rstable['zip']==zipcode,'zri_test']=globals()[f"scaler_{zipcode}"].inverse_transform(rstable_filtered[['zri_test']])
    rstable.loc[rstable['zip']==zipcode,'zri_predicted']=globals()[f"scaler_{zipcode}"].inverse_transform(rstable_filtered[['zri_predicted']])  
rstable.loc[:,'zri_test'] = np.exp(rstable.loc[:,'zri_test'])
rstable.loc[:,'zri_predicted'] = np.exp(rstable.loc[:,'zri_predicted'])
rstable.isna().sum()

zip              0
zri_test         0
zri_predicted    0
dtype: int64

In [86]:
#RESULTS 
r2 = r2_score(rstable['zri_test'],rstable['zri_predicted'])
rmse = sqrt(mean_squared_error(rstable['zri_test'],rstable['zri_predicted']))
print(f'R2: {r2*100}')
print(f'RMSE: {rmse}')

R2: 98.71343352758063
RMSE: 75.46409446867541


In [87]:
#train_X.columns

Index(['income_per_capita', 'percent_buildings_10_19_units', 'total_pop',
       'median_age', 'percent_workforce_unemployed',
       'percent_commute_public_transport', 'percent_associates',
       'percent_highschool', 'percent_native_am', 'percent_white',
       'percent_rental_units_occupied', 'percent_buildings_20_49_units',
       'percent_units_owner_occupied', 'percent_buildings_50+_units',
       'percent_work_from_home', 'poverty_rate', 'median_building_age',
       'zri_lag1', 'zri_lag2', 'zri_lag3', 'zri_lag4', 'zri_lag5', 'zri_lag6',
       'zri_lag7', 'zri_lag8', 'zri_lag9', 'zri_lag10', 'zri_lag11',
       'zri_diff_lag1_lag12', 'income_per_capita_diff_lag1_lag12',
       'income_per_capita_lag1', 'income_per_capita_lag6',
       'income_per_capita_lag12',
       'percent_buildings_10_19_units_diff_lag1_lag12',
       'percent_buildings_10_19_units_lag1',
       'percent_buildings_10_19_units_lag6',
       'percent_buildings_10_19_units_lag12', 'total_pop_diff_lag1_lag12

In [88]:
importance = np.abs(lasso_best.coef_)
coef = lasso_best.coef_
feature_names = train_X.columns
feature_importances= pd.DataFrame([feature_names,importance,coef]).T
feature_importances.columns = ['feature_names','coef_abs','coef']

feature_importances = feature_importances.sort_values(by='coef_abs',ascending=False)

In [89]:
feature_importances['Parent_feature']=feature_importances['feature_names'].apply(lambda x: x[0:x.find('_lag')] if x.find('_lag')!=-1 else x)
feature_importances['Parent_feature']=feature_importances['Parent_feature'].apply(lambda x: x[0:x.find('_diff')] if x.find('_diff')!=-1 else x)

In [90]:
#feature_importances=feature_importances.groupby('Parent_feature').agg('mean').sort_values('Importance',ascending=False)
#feature_importances

,feature_names,coef_abs,coef,Parent_feature
17,zri_lag1,1.492442,1.492442,zri
18,zri_lag2,0.598477,-0.598477,zri
28,zri_diff_lag1_lag12,0.053407,0.053407,zri
19,zri_lag3,0.048796,-0.048796,zri
27,zri_lag11,0.03075,0.03075,zri
23,zri_lag7,0.020821,0.020821,zri
26,zri_lag10,0.017657,0.017657,zri
20,zri_lag4,0.013095,0.013095,zri
47,percent_workforce_unemployed_lag6,0.011875,0.011875,percent_workforce_unemployed
46,percent_workforce_unemployed_lag1,0.011182,-0.011182,percent_workforce_unemployed


In [91]:
feats_agg = feature_importances[['coef_abs','Parent_feature']]
feats_agg['coef_abs'] = feats_agg['coef_abs'].astype('float')
feats_agg_abs = feats_agg.groupby('Parent_feature').agg('mean').sort_values('coef_abs',ascending=False)
feats_agg_abs

<ipython-input-91-0836822478da>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feats_agg['coef_abs'] = feats_agg['coef_abs'].astype('float')


,coef_abs
Parent_feature,
zri,0.190395
percent_workforce_unemployed,0.004869
median_building_age,0.003706
income_per_capita,0.003297
total_pop,0.002309
percent_white,0.001529
poverty_rate,0.000909
percent_native_am,0.000872
percent_units_owner_occupied,0.000785


In [92]:
feats_agg = feature_importances[['coef','Parent_feature']]
feats_agg['coef'] = feats_agg['coef'].astype('float')
feats_agg_sign = feats_agg.groupby('Parent_feature').agg('mean').sort_values('coef',ascending=False)
feats_agg_sign

<ipython-input-92-13305d22957c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feats_agg['coef'] = feats_agg['coef'].astype('float')


,coef
Parent_feature,
zri,0.082516
median_building_age,0.002537
poverty_rate,0.000909
income_per_capita,0.000660
percent_work_from_home,0.000553
median_age,0.000518
percent_rental_units_occupied,0.000287
total_pop,0.000263
percent_associates,0.000243


In [93]:
title = '02_predicted_Final_Auto_ACS'
fi = title+'_FI'
fi2 = title+'_FI_Agg'
rstable.to_csv(f'../../../zillow_orientation/Residuals/{title}.csv')
feature_importances.to_csv(f'../../../zillow_orientation/Residuals/{fi}.csv')
feats_agg_abs.to_csv(f'../../../zillow_orientation/Residuals/{fi2}.csv')

In [ ]:
# BEST AUTO + ACS 
# Lasso(alpha=0.0001)
# R2: 98.71343352758063
# RMSE: 75.46409446867541


















# 63 features total 
# ['percent_buildings_50+_units', 'percent_associates',
#        'percent_rental_units_occupied', 'percent_white', 'percent_highschool',
#        'percent_work_from_home', 'percent_buildings_20_49_units',
#        'median_building_age', 'median_age', 'percent_commute_public_transport',
#        'percent_buildings_10_19_units', 'income_per_capita',
#        'percent_native_am', 'percent_workforce_unemployed', 'poverty_rate',
#        'percent_units_owner_occupied', 'total_pop', 'zri_lag1', 'zri_lag2',
#        'zri_lag3', 'zri_lag4', 'zri_lag5', 'zri_lag6', 'zri_lag7', 'zri_lag8',
#        'zri_lag9', 'zri_lag10', 'zri_lag11', 'zri_diff_lag1_lag12',
#        'percent_buildings_50+_units_diff_lag1_lag12',
#        'percent_buildings_50+_units_lag12',
#        'percent_associates_diff_lag1_lag12', 'percent_associates_lag12',
#        'percent_rental_units_occupied_diff_lag1_lag12',
#        'percent_rental_units_occupied_lag12', 'percent_white_diff_lag1_lag12',
#        'percent_white_lag12', 'percent_highschool_diff_lag1_lag12',
#        'percent_highschool_lag12', 'percent_work_from_home_diff_lag1_lag12',
#        'percent_work_from_home_lag12',
#        'percent_buildings_20_49_units_diff_lag1_lag12',
#        'percent_buildings_20_49_units_lag12',
#        'median_building_age_diff_lag1_lag12', 'median_building_age_lag12',
#        'median_age_diff_lag1_lag12', 'median_age_lag12',
#        'percent_commute_public_transport_diff_lag1_lag12',
#        'percent_commute_public_transport_lag12',
#        'percent_buildings_10_19_units_diff_lag1_lag12',
#        'percent_buildings_10_19_units_lag12',
#        'income_per_capita_diff_lag1_lag12', 'income_per_capita_lag12',
#        'percent_native_am_diff_lag1_lag12', 'percent_native_am_lag12',
#        'percent_workforce_unemployed_diff_lag1_lag12',
#        'percent_workforce_unemployed_lag12', 'poverty_rate_diff_lag1_lag12',
#        'poverty_rate_lag12', 'percent_units_owner_occupied_diff_lag1_lag12',
#        'percent_units_owner_occupied_lag12', 'total_pop_diff_lag1_lag12',
#        'total_pop_lag12']

# R2: 98.68438523437717
# RMSE: 76.31125919884467
# Lasso(alpha=1e-05)





# 3. ZRI + ALL 

In [119]:
df_zri_all = df_scaled_no_lags 

In [120]:
# def lag_gen(df):
#     for i in range(1,12):
#         df=laggenerator(i, 'zri', df)
#     df=laggenerator_diff12(1, 'zri', df)
#     return df

In [121]:
#ZRI
df_zri_all = lag_gen(df_zri_all)

In [122]:
#ACS
for col in acs_cols_keep:
    df_zri_acs = laggenerator_diff12(1,col,df_zri_acs)
    df_zri_acs = laggenerator(1,col,df_zri_acs)
    df_zri_acs = laggenerator(6,col,df_zri_acs)
    df_zri_acs = laggenerator(12,col,df_zri_acs)

In [123]:
#BIKESHARE AND ECONOMIC 
for col in bikeshare_cols_keep + economic_cols_keep:
    df_zri_all = laggenerator(1, col, df_zri_all)
    df_zri_all = laggenerator_diff12(1, col, df_zri_all)

In [124]:
#TRENDS 
for col in trends_cols_keep:
    for i in range(1,7):
        df_zri_all=laggenerator(i, col, df_zri_all)
        df_zri_all=laggenerator_diff(i, col, df_zri_all)
    df_zri_all = laggenerator_diff12(1,col,df_zri_all)    

In [125]:
df_zri_all = df_zri_all.fillna(0)

In [126]:
df = df_zri_all
train = df[(df['year'] < 2019) & (df['year']>=2015)]
test = df[df['year']==2019]
save_train = train.copy()
save_test = test.copy()

train = train.drop(['zip','City','State','Metro','CountyName','year','month','year-month'],axis=1)
train_y = train['zri']
train_X = train.drop(['zri'],axis=1)

test_y = test['zri']
test_X = test
#test_X = test.drop(['zri'],axis=1)

In [ ]:
#Lasso Grid Search
lasso = Lasso()
grid = dict()
grid['alpha'] = [1e-15, 1e-10, 1e-8, 1e-5,1e-4, 1e-3,1e-2, 1, 5, 10]
lasso_grid = GridSearchCV(lasso, grid, cv=5, n_jobs=-1).fit(train_X,train_y)
lasso_best = lasso_grid.best_estimator_
lasso_best.fit(train_X, train_y)


In [ ]:
scaled_predictions_y = X_test['zri']
temp = pd.concat([save_test['zip'],test_y],axis=1)
temp.reset_index(drop=True, inplace=True)
scaled_predictions_y.reset_index(drop=True, inplace=True)
rstable = pd.concat([temp,scaled_predictions_y],axis=1)
rstable.columns = ['zip','zri_test','zri_predicted']
rstable

In [ ]:
for zipcode in rstable['zip'].unique():
    rstable_filtered=rstable[rstable['zip']==zipcode]
    rstable.loc[rstable['zip']==zipcode,'zri_test']=globals()[f"scaler_{zipcode}"].inverse_transform(rstable_filtered[['zri_test']])
    rstable.loc[rstable['zip']==zipcode,'zri_predicted']=globals()[f"scaler_{zipcode}"].inverse_transform(rstable_filtered[['zri_predicted']])  
rstable.loc[:,'zri_test'] = np.exp(rstable.loc[:,'zri_test'])
rstable.loc[:,'zri_predicted'] = np.exp(rstable.loc[:,'zri_predicted'])
rstable.isna().sum()

In [ ]:
#RESULTS 
r2 = r2_score(rstable['zri_test'],rstable['zri_predicted'])
rmse = sqrt(mean_squared_error(rstable['zri_test'],rstable['zri_predicted']))
print(f'R2: {r2*100}')
print(f'RMSE: {rmse}')